# Step 2 - Option Market Making

In this notebook, we use the previously built valuation to create a simple option market making strategy. 

## 1 Standard Imports

Don't change these. If you need custom imports than you need to put them directly into your strategy.

In [2]:
from dataclasses import dataclass
import datetime as dt
import numpy as np
from typing import Tuple
from trading_simulation.book_history import Snapshot, Trade
from trading_simulation.simulation import HARD_RISK_LIMITS, SOFT_RISK_LIMITS, PositionStats, TradingSimulation, SimulationSettings
from trading_simulation.strategy import Strategy, Valuation, Execution

## 2 Strategy Interface

### 2.1 Execution

Remember the structure of the `Execution` class:

```python
@dataclass(frozen=True)
class Execution:
    futures_volume: int = 0
    option_volume: int = 0
```

As part of response to `handle_option_trade`, the two fields have the following interpretation:

- `option_volume` is the signed volume share of the trade that you would like to participate in. The volume cannot be larger than the `traded_volume.` Assume e.g. a trade `Trade(traded_price=10.0, traded_volume=100)`. Replying with `option_volume = -10` means that you will sell 10 lots at 10 EUR.
- `futures_volume` is the signed volume in the futures that you would like to trade. As a response to the an option trade, this means that you have to "cross" the order book in the futures. I.e. a buy (sell) with positive (negative) `futures_volume` will be executed against the then-current best-bid (best-ask) price of the futures contract. The traded volume cannot be higher than the top-level volume on the respective side in the order book.

# 3 Implementing Options Market Making

We now implement a simple market making strategy. Whenever a trade happens above (below) our valuation, we sell (buy) the full trade volume. We do not incorporate a delta hedge yet and disregard any risk limits.

In [266]:
import math

@dataclass
class OptionMarketMaking(Strategy):
    option_value: float = 10.0
    stats: PositionStats = None

    def handle_option_snapshot(self, snapshot: Snapshot) -> Valuation:
        self.option_value = self._calculate_mid(snapshot)
        return self.valuation

    def handle_option_trade(self, trade: Trade) -> Tuple[Valuation, Execution]:
        option_volume = trade.traded_volume * (-1 if trade.traded_price > self.option_value else 1) 
        return self.valuation, Execution(option_volume=option_volume)
    
    def _calculate_mid(self, snapshot: Snapshot) -> float:
        return 0.5 * (snapshot.bid_price + snapshot.ask_price)
        
    @property
    def valuation(self) -> Valuation:
        return Valuation(option_price=self.option_value)
    
@dataclass
class BDE(Strategy):
    delta_limit = 41
    vega_limit = 10000
    option_value: float = 10.0
    futures_value: float = 10.0
    option_delta = 0.
    option_vega = 0.
    futures_delta = 0
    futures_vega = 0
    stats: PositionStats = None
        
#     def __init__(self, delta_limit):
#         self.delta_limit = delta_limit
#         self.vega_limit = 10000
#         self.option_value: float = 10.0
#         self.futures_value: float = 10.0
#         self.option_delta = 0.
#         self.option_vega = 0.
#         self.futures_delta = 0
#         self.futures_vega = 0
#         stats: PositionStats = None

    def handle_option_snapshot(self, snapshot: Snapshot) -> Valuation:
        self.option_value = self._calculate_mid(snapshot)
        self.option_delta = snapshot.delta
        self.option_vega = snapshot.vega
        return self.valuation

    def handle_option_trade(self, trade: Trade) -> Tuple[Valuation, Execution]:
        option_volume = 0
        gains = abs(trade.traded_price - self.option_value)*10
        for i in reversed(range(0,trade.traded_volume+1)):
            value1 = i*self.option_delta * (-1 if trade.traded_price > self.option_value else 1)+self.stats.delta
            value2 = i*self.option_vega * (-1 if trade.traded_price > self.option_value else 1)+self.stats.vega
            if abs(value1) <= self.delta_limit and abs(value2) <= 7500:
                option_volume = int(i*(-1 if trade.traded_price > self.option_value else 1)*(gains/math.sqrt(1+gains**2)))
                break
        return self.valuation, Execution(option_volume=option_volume)
    
    def _calculate_mid(self, snapshot: Snapshot) -> float:
#         return 0.5*(snapshot.bid_price + snapshot.ask_price)
        return (snapshot.bid_price*snapshot.ask_volume + snapshot.ask_price*snapshot.bid_volume)/((snapshot.bid_volume+snapshot.ask_volume))

    def handle_futures_snapshot(self, snapshot: Snapshot) -> Valuation:
        self.futures_value = self._calculate_mid(snapshot)
        self.futures_delta = snapshot.delta
        self.futures_vega = snapshot.delta
        return Valuation()
    
    def handle_futures_trade(self, trade: Trade) -> Tuple[Valuation, Execution]:
        futures_volume = 0
        for i in reversed(range(0,trade.traded_volume+1)):
            value1 = i*self.futures_delta * (-1 if trade.traded_price > self.futures_value else 1)+self.stats.delta
            value2 = i*self.futures_vega * (-1 if trade.traded_price > self.futures_value else 1)+self.stats.vega
            if abs(value1) <= 10 and abs(value2) <= 7500:
                futures_volume = i*(-1 if trade.traded_price > self.futures_value else 1)
                break
            elif (abs(self.stats.delta) > 50) and (abs(value1) < abs(self.stats.delta)):
                futures_volume = i*(-1 if trade.traded_price > self.futures_value else 1)
                break
                
        return self.valuation, Execution(futures_volume=futures_volume)

    def handle_position_stats(self, stats: PositionStats) -> None:
        self.stats = stats
        #print(self.stats.delta)
        pass
        
    """futures_position: int
    option_position: int
    cash_balance: float
    profit: float
    delta: float
    vega: float"""
        
    @property
    def valuation(self) -> Valuation:
        return Valuation(option_price=self.option_value, futures_price=self.futures_value)

## 4 Evaluating the Trading Strategy

### 4.1 Running the Simulation

As we don't try to stay withing the soft delta (50 futures) or vega (10,000 EUR) limits, we set `ignore_limits = True` in the `SimulationSettings`. Otherwise, we would get an error when our position breaches either limit. See below for details on how these limits work.

In [ ]:
strategies = {
    'simple market making': BDE()
}
simulation = TradingSimulation()
settings = SimulationSettings(max_count=2000000, ignore_limits=False)
result = simulation.simulate(strategies, settings)

In [205]:
# for i in range(45,50):
#     strategies = {
#         'simple market making': BDE(i),
#     }
#     simulation = TradingSimulation()
#     settings = SimulationSettings(max_count=100000, ignore_limits=True)
#     result = simulation.simulate(strategies, settings)
#     print(result)

In [264]:
result

simple market making
--------------------
profit & loss: 47,080.90
sharpe ratio: 5,782.17
number of futures trades: 12,283
number of option trades: 919
traded futures volume: 83,612
traded options volume: 49,443
max. delta exposure: 45.92
max. vega exposure: -7,524.76
errors: []

### 4.2 Plotting the Results

Apart from plotting the valuation, which did not change, we can now also plot our Greeks as well as profit & loss.

In [182]:
result.plot_valuations(include_futures=True, include_option=False, count=1000, start_time=dt.time(9, 0, 0), end_time=dt.time(17, 30, 0))

In [183]:
result.plot_greeks(count=1000, start_time=dt.time(9, 0, 0), end_time=dt.time(17, 30, 0))

In [35]:
result.plot_profits(count=1000, start_time=dt.time(9, 0, 0), end_time=dt.time(17, 30, 0))

## 5 Strategy Interface

Your custom strategies can also react to futures `Trade` and `Snapshot` events, e.g. for delta hedging. The full `Strategy` interface is given by:

```python
class Strategy:
    def handle_option_snapshot(self, snapshot: Snapshot) -> Valuation:
        return Valuation()

    def handle_option_trade(self, trade: Trade) -> Tuple[Valuation, Execution]:
        return Valuation(), Execution()
    
    def handle_futures_snapshot(self, snapshot: Snapshot) -> Valuation:
        return Valuation()

    def handle_futures_trade(self, trade: Trade) -> Tuple[Valuation, Execution]:
        return Valuation(), Execution()
    
    def handle_position_stats(self, stats: PositionStats) -> None:
        pass
```

`handle_position_stats` returns information about your current positions and Greeks. You should generally maintain these yourself but using the callback is useful to debug your implementation.

```python
@dataclass(frozen=True)
class PositionStats:
    futures_position: int
    option_position: int
    cash_balance: float
    profit: float
    delta: float
    vega: float
```

## 6 Risk Limits

There are two types of risk limits: `SOFT_RISK_LIMITS` and `HARD_RISK_LIMITS`. Both of them define a delta and vega limit. They are

| type              | delta | vega   |
| :---------------- | :---: | :----: |
| `SOFT_RISK_LIMIT` | 50    | 10,000 |
| `HARD_RISK_LIMIT` | 100   | 11,000 |

Breaching a hard limit means that your strategy gets disqualified. Breaching a soft limit incurs a penalty of 10 EUR per second of breach.

## 7 Next Steps

Think about how you could improve the simple trading strategy to be as profitable as possible. Consider the following hints:

1. How do you ensure not to breach your risk limits?
2. The binding risk limit seems to be the delta. Could you improve your trading by hedging your delta exposure by trading in the futures?
3. The delta and vega of the option change over time. It is wise to exhaust the full limit or instead leave a buffer?
4. Since risk limits constrain you in how much delta and vega exposure you can build - is every trade opportunity equally desirable?
5. The futures is much more liquid than the option. Are there opportunities in aggressively trading the option on large futures moves?

## 8 Submission

Use

```python
Strategy.save(AwesomeStrategy, 'name_of_your_group')
```

to save your submission. Note that your strategy needs to be self-contained. This means:

- You cannot import additional modules outside the `handle_` functions.
- You cannot reference global constants, functions or other classes.

In [265]:
Strategy.save(BDE, 'group12')